In [1]:
from datasets import load_dataset

In [2]:
eli5 = load_dataset("eli5", split="train_asks[:5000]")

Found cached dataset eli5 (C:/Users/mlach/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa)


In [3]:
eli5 = eli5.train_test_split(test_size=0.2)

In [4]:
eli5["train"][0]

{'q_id': '11dbtl',
 'title': 'Is it possible to add enough salt (or other solute) to water such that it is still an aqueous solution, but dense enough for a man to walk on?',
 'selftext': "Pretty much the title. If you added too much solute, you'd be walking on a wet, mucky solid I would think. Is it possible to add so much solute that the aqueous solution is still liquid but dense enough to support a grown man (say 150 pounds over a surface area of  1ft^2)",
 'document': '',
 'subreddit': 'askscience',
 'answers': {'a_id': ['c6lgnu1', 'c6ll34v'],
  'text': ['Corn starch, but only if you run.  A supersaturated solution will become solid under sudden stress, but remain liquid otherwise.',
   "It would be supporting you by buoyancy, so the amount of liquid you displace would have to be equal to your weight to keep you up. Normally 150 pounds of water takes up 68 liters of space. In order for the water to stay below your ankles, you'd only be using your feet which take up maybe 2 liters a

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model_checkpoint = "distilgpt2"

In [6]:
tokenizer

GPT2TokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'})

In [7]:
eli5 = eli5.flatten()

eli5["train"][0]

{'q_id': '11dbtl',
 'title': 'Is it possible to add enough salt (or other solute) to water such that it is still an aqueous solution, but dense enough for a man to walk on?',
 'selftext': "Pretty much the title. If you added too much solute, you'd be walking on a wet, mucky solid I would think. Is it possible to add so much solute that the aqueous solution is still liquid but dense enough to support a grown man (say 150 pounds over a surface area of  1ft^2)",
 'document': '',
 'subreddit': 'askscience',
 'answers.a_id': ['c6lgnu1', 'c6ll34v'],
 'answers.text': ['Corn starch, but only if you run.  A supersaturated solution will become solid under sudden stress, but remain liquid otherwise.',
  "It would be supporting you by buoyancy, so the amount of liquid you displace would have to be equal to your weight to keep you up. Normally 150 pounds of water takes up 68 liters of space. In order for the water to stay below your ankles, you'd only be using your feet which take up maybe 2 liters

In [8]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]], truncation=True)

In [9]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=False,
    remove_columns=eli5["train"].column_names,
)

  0%|          | 0/4000 [00:00<?, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [10]:
block_size = 128


def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [11]:
lm_dataset = tokenized_eli5.map(group_texts, batched=False)

  0%|          | 0/4000 [00:00<?, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [12]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, return_tensors="tf")

In [13]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [14]:
from transformers import TFAutoModelForCausalLM

model = TFAutoModelForCausalLM.from_pretrained("distilgpt2")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at distilgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [17]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tf_train_set = model.prepare_tf_dataset(
    lm_dataset["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    lm_dataset["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

IndexError: tuple index out of range

In [18]:
import tensorflow as tf

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.
